In [1]:
# Importazione delle librerie necessarie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.linear_model import Lasso,Ridge
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
#to display all the graph in the workbook
sns.set_style("whitegrid",{'grid.linestyle': '--'})
from sklearn.ensemble import BaggingRegressor
from sklearn.datasets import make_regression


# Upload del csv + display
path = "https://raw.githubusercontent.com/lucaonnis25/dati/main/SeoulBikeData.csv"
df = pd.read_csv(path, encoding='latin-1', sep=",")
display(df)

Date  Rented Bike Count  Hour  Temperature(°C)  Humidity(%)  \
0     01/12/2017                254     0             -5.2           37   
1     01/12/2017                204     1             -5.5           38   
2     01/12/2017                173     2             -6.0           39   
3     01/12/2017                107     3             -6.2           40   
4     01/12/2017                 78     4             -6.0           36   
...          ...                ...   ...              ...          ...   
8755  30/11/2018               1003    19              4.2           34   
8756  30/11/2018                764    20              3.4           37   
8757  30/11/2018                694    21              2.6           39   
8758  30/11/2018                712    22              2.1           41   
8759  30/11/2018                584    23              1.9           43   

      Wind speed (m/s)  Visibility (10m)  Dew point temperature(°C)  \
0                  2.2              2000                      -17.6   
1                  0.8              2000                      -17.6   
2                  1.0              2000                      -17.7   
3                  0.9              2000                      -17.6   
4                  2.3              2000                      -18.6   
...                ...               ...                        ...   
8755               2.6              1894                      -10.3   
8756               2.3              2000                       -9.9   
8757               0.3              1968                       -9.9   
8758               1.0              1859                       -9.8   
8759               1.3              1909                       -9.3   

      Solar Radiation (MJ/m2)  Rainfall(mm)  Snowfall (cm) Seasons  \
0                         0.0           0.0            0.0  Winter   
1                         0.0           0.0            0.0  Winter   
2                         0.0           0.0            0.0  Winter   
3                         0.0           0.0            0.0  Winter   
4                         0.0           0.0            0.0  Winter   
...                       ...           ...            ...     ...   
8755                      0.0           0.0            0.0  Autumn   
8756                      0.0           0.0            0.0  Autumn   
8757                      0.0           0.0            0.0  Autumn   
8758                      0.0           0.0            0.0  Autumn   
8759                      0.0           0.0            0.0  Autumn   

         Holiday Functioning Day  
0     No Holiday             Yes  
1     No Holiday             Yes  
2     No Holiday             Yes  
3     No Holiday             Yes  
4     No Holiday             Yes  
...          ...             ...  
8755  No Holiday             Yes  
8756  No Holiday             Yes  
8757  No Holiday             Yes  
8758  No Holiday             Yes  
8759  No Holiday             Yes  

[8760 rows x 14 columns]

In [2]:
# converting date column dtype object to date
df['Date']=pd.to_datetime(df['Date'])

# split day of week, month and year in three column
df['day_of_week'] = df['Date'].dt.day_name() # extract week name from Date column
df["month"] = df['Date'].dt.month_name()   # extract month name from Date column
df["year"] = df['Date'].map(lambda x: x.year).astype("object")

# drop the Date column
df.drop(columns=['Date'],inplace=True)

# convert Hour column integer to Categorical
df['Hour']=df['Hour'].astype('object')


# Divide Data in categorical and numerical features
numeric_features= df.select_dtypes(exclude='object')
categorical_features=df.select_dtypes(include='object')


numeric_features.head()
categorical_features.head()

Hour Seasons     Holiday Functioning Day day_of_week    month  year
0    0  Winter  No Holiday             Yes    Thursday  January  2017
1    1  Winter  No Holiday             Yes    Thursday  January  2017
2    2  Winter  No Holiday             Yes    Thursday  January  2017
3    3  Winter  No Holiday             Yes    Thursday  January  2017
4    4  Winter  No Holiday             Yes    Thursday  January  2017

In [3]:
# checking categorical column value count
for i in categorical_features.columns:
  print("\n ")
  print('column name  : ', i)
  print(df[i].value_counts())


 
column name  :  Hour
0     365
1     365
22    365
21    365
20    365
19    365
18    365
17    365
16    365
15    365
14    365
13    365
12    365
11    365
10    365
9     365
8     365
7     365
6     365
5     365
4     365
3     365
2     365
23    365
Name: Hour, dtype: int64

 
column name  :  Seasons
Spring    2208
Summer    2208
Autumn    2184
Winter    2160
Name: Seasons, dtype: int64

 
column name  :  Holiday
No Holiday    8328
Holiday        432
Name: Holiday, dtype: int64

 
column name  :  Functioning Day
Yes    8465
No      295
Name: Functioning Day, dtype: int64

 
column name  :  day_of_week
Sunday       1296
Wednesday    1272
Tuesday      1272
Thursday     1248
Saturday     1248
Friday       1224
Monday       1200
Name: day_of_week, dtype: int64

 
column name  :  month
January      744
March        744
May          744
July         744
August       744
October      744
December     744
April        720
June         720
September    720
November     720
Februar

In [4]:
# Converting days of weeks in Two variable from Monaday to Friday in Weekdays and Saturday and Sunday to Weekend
df['week'] = df['day_of_week'].apply(lambda x:'Weekend'  if x=='Saturday' or  x== 'Sunday' else 'Weekdays')

In [5]:
# value counts of Week column
df.week.value_counts()

Weekdays    6216
Weekend     2544
Name: week, dtype: int64

In [6]:
# droping the days of week column from df and from categorical feature
df.drop(columns=['day_of_week'], inplace=True)
categorical_features.drop(columns=['day_of_week'], inplace=True)

In [7]:
# Correlation with Rented Bike Count
df.corr()['Rented Bike Count']

Rented Bike Count            1.000000
Temperature(°C)              0.538558
Humidity(%)                 -0.199780
Wind speed (m/s)             0.121108
Visibility (10m)             0.199280
Dew point temperature(°C)    0.379788
Solar Radiation (MJ/m2)      0.261837
Rainfall(mm)                -0.123074
Snowfall (cm)               -0.141804
Name: Rented Bike Count, dtype: float64

In [8]:

# using seaborn heatmap for ploting correlation graph
plt.figure(figsize=(10,8))
sns.heatmap(abs(df.corr()), cmap='coolwarm', annot=True)

<Axes: >

In [9]:
df.drop(columns=['Dew point temperature(°C)'],inplace=True)

In [10]:
new_df=pd.get_dummies(df,drop_first=True,sparse=True)
new_df.head(2)

Rented Bike Count  Temperature(°C)  Humidity(%)  Wind speed (m/s)  \
0                254             -5.2           37               2.2   
1                204             -5.5           38               0.8   

   Visibility (10m)  Solar Radiation (MJ/m2)  Rainfall(mm)  Snowfall (cm)  \
0              2000                      0.0           0.0            0.0   
1              2000                      0.0           0.0            0.0   

   Hour_1  Hour_2  ...  month_January  month_July  month_June  month_March  \
0       0       0  ...              1           0           0            0   
1       1       0  ...              1           0           0            0   

   month_May  month_November  month_October  month_September  year_2018  \
0          0               0              0                0          0   
1          0               0              0                0          0   

   week_Weekend  
0             0  
1             0  

[2 rows x 49 columns]

In [11]:
X = new_df.drop(columns='Rented Bike Count')
y = np.sqrt(new_df['Rented Bike Count'])

In [12]:
# Train test split our data
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25,random_state=42)

In [13]:
y_train.head()

2223    35.185224
798     11.704700
7752    29.849623
734     16.186414
2881    22.405357
Name: Rented Bike Count, dtype: float64

In [14]:
y_test.head()

6056    41.569219
5556    28.670542
5990    25.651511
7674    52.115257
3319    32.908965
Name: Rented Bike Count, dtype: float64

In [15]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6570, 48), (2190, 48), (6570,), (2190,))

In [16]:
def score_metrics(actual,predicted):
  mae = mean_absolute_error(actual**2,predicted**2)
  print('mean absolute error is  :',mae)
  mse = mean_squared_error(actual**2,predicted**2)
  print('mean squared error is  :',mse)
  print('Root mean squared error is  :',np.sqrt(mse))
  print("R2 score is  :",r2_score(actual**2,predicted**2))

In [17]:
yeo = PowerTransformer()
X_train = yeo.fit_transform(X_train)
X_test = yeo.transform(X_test)

In [18]:
# Fitting the model
regression = LinearRegression()
regression.fit(X_train,y_train)
regression.score(X_train,y_train)

0.7948131221837624

In [19]:
#predicting
reg_pred = regression.predict(X_test)

In [20]:
score_metrics(y_test,reg_pred)

mean absolute error is  : 207.4166678402189
mean squared error is  : 93868.92036183795
Root mean squared error is  : 306.38035244094544
R2 score is  : 0.7706043626064047


In [21]:
 regression.coef_

array([ 4.84244127, -1.46299861, -0.10044714,  0.29088452,  3.15181295,
       -2.97282855,  0.017911  , -0.43657148, -1.06081323, -1.41706587,
       -2.02154163, -1.92712698, -0.95802639,  0.01025612,  0.86074665,
       -0.51390573, -1.51276992, -1.57296373, -1.37571477, -1.34499939,
       -1.29350974, -1.09078416, -0.68059033,  0.28889087,  1.67507948,
        1.32330204,  1.31495447,  1.37700904,  1.00303563,  0.34014356,
       -1.32715513, -1.38178009, -3.39349514,  0.60842228,  5.30792837,
       -0.2321773 ,  0.00764478, -0.20277928, -0.01720571, -0.28303424,
        0.95185455, -0.17567796,  0.37627942,  0.06519845,  0.35172126,
       -0.04266782, -0.57320668, -0.39958343])

In [22]:
regression.intercept_

23.442819487037088

In [23]:
L1 = Lasso(alpha=0.001, max_iter=1000)
L1.fit(X_train,y_train)
L1.score(X_train,y_train)

0.7948120063359843

In [24]:
L1_pred = L1.predict(X_test)
score_metrics(y_test,L1_pred)

mean absolute error is  : 207.46370269633783
mean squared error is  : 93937.02951497176
Root mean squared error is  : 306.4914835928916
R2 score is  : 0.7704379183505717


In [25]:
L1.coef_

array([ 4.84071092, -1.46449278, -0.09793591,  0.29092238,  3.125142  ,
       -2.97414267,  0.01635459, -0.43022366, -1.05423021, -1.41057138,
       -2.01497009, -1.9206802 , -0.95127885,  0.01658726,  0.86995557,
       -0.49953006, -1.49646076, -1.55583967, -1.35785492, -1.32762605,
       -1.27633522, -1.07402515, -0.66455624,  0.30066554,  1.6843267 ,
        1.32963752,  1.31934944,  1.38111968,  1.00720702,  0.34445351,
       -1.32145114, -1.37418404, -3.39204788,  0.60724315,  5.30609398,
       -0.23111552,  0.0079985 , -0.20123441, -0.01618454, -0.2807969 ,
        0.95169734, -0.17483395,  0.37632458,  0.06544604,  0.35304664,
       -0.03972067, -0.57093445, -0.39848292])

In [26]:
L1.intercept_

23.442819487037088

In [27]:
lasso = Lasso()
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100,0.0014]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=5)
lasso_regressor.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [1e-15, 1e-13, 1e-10, 1e-08, 1e-05, 0.0001,
                                   0.001, 0.01, 0.1, 1, 5, 10, 20, 30, 40, 45,
                                   50, 55, 60, 100, 0.0014]},
             scoring='neg_mean_squared_error')

In [28]:
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("\nUsing ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

The best fit alpha value is found out to be : {'alpha': 0.0014}

Using  {'alpha': 0.0014}  the negative mean squared error is:  -32.63209718317992


In [29]:
lasso_pred = lasso_regressor.predict(X_test)
score_metrics(y_test,lasso_pred)

mean absolute error is  : 207.48363096606084
mean squared error is  : 93964.52841872428
Root mean squared error is  : 306.53634110611466
R2 score is  : 0.7703707168899646


In [30]:
L2 = Ridge()
parameters = {'alpha': [1e-15,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1,5,10,20,30,40,45,50,55,60,100,0.5,1.5,1.6,1.7,1.8,1.9]}
L2_regressor = GridSearchCV(L2, parameters, scoring='neg_mean_squared_error', cv=5)
L2_regressor.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 1e-05, 0.0001, 0.001,
                                   0.01, 1, 5, 10, 20, 30, 40, 45, 50, 55, 60,
                                   100, 0.5, 1.5, 1.6, 1.7, 1.8, 1.9]},
             scoring='neg_mean_squared_error')

In [31]:
print("The best fit alpha value is found out to be :" ,L2_regressor.best_params_)
print("\nUsing ",L2_regressor.best_params_, " the negative mean squared error is: ", L2_regressor.best_score_)

The best fit alpha value is found out to be : {'alpha': 1.9}

Using  {'alpha': 1.9}  the negative mean squared error is:  -32.63429585000174


In [32]:
from sklearn.preprocessing import PolynomialFeatures

In [33]:
poly = PolynomialFeatures()
poly_X_train = poly.fit_transform(X_train)
poly_X_test = poly.transform(X_test)

poly_X_train[:5]
poly_X_test[:5]


regression.fit(poly_X_train,y_train)

regression.score(poly_X_train,y_train)

poly_pred = regression.predict(poly_X_test)

score_metrics(y_test,poly_pred)

mean absolute error is  : 132.84926692385855
mean squared error is  : 47689.53388962669
Root mean squared error is  : 218.37933485022498
R2 score is  : 0.8834569420693806


In [55]:
df.head(2)

Rented Bike Count Hour  Temperature(°C)  Humidity(%)  Wind speed (m/s)  \
0                254    0             -5.2           37               2.2   
1                204    1             -5.5           38               0.8   

   Visibility (10m)  Solar Radiation (MJ/m2)  Rainfall(mm)  Snowfall (cm)  \
0              2000                      0.0           0.0            0.0   
1              2000                      0.0           0.0            0.0   

  Seasons     Holiday Functioning Day    month  year      week  
0  Winter  No Holiday             Yes  January  2017  Weekdays  
1  Winter  No Holiday             Yes  January  2017  Weekdays

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Supponendo che il tuo DataFrame sia "Seoul_Bike_Renting"
# Sostituisci 'nome_colonna_target' con il nome della colonna target che vuoi prevedere

# Seleziona solo le colonne numeriche per la suddivisione e la normalizzazione
features = df.drop(columns=['Rented Bike Count', 'Seasons', 'Holiday', 'Functioning Day', 'month', 'year', 'week']) #, 'Date', 'Seasons', 'Holiday', 'Functioning Day'])
target = df['Rented Bike Count']

# Suddivisione del DataFrame in set di addestramento, convalida e test
X_train, X_temp, y_train, y_temp = train_test_split(
    features,
    target,
    test_size=0.7,  # Percentuale per il set di addestramento
    random_state=42  # Seed per la riproducibilità
)

x_valid, x_test, y_valid, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,  # Percentuale per il set di convalida e test
    random_state=42  # Seed per la riproducibilità
)

# Normalizzazione dei dati usando la standardizzazione (z-score)
scaler = StandardScaler()
x_train_normalized = scaler.fit_transform(X_train)
x_valid_normalized = scaler.transform(x_valid)
x_test_normalized = scaler.transform(x_test)


In [58]:
import keras

# Sequential model is composed by a sequential
# single stack of layers connected sequentially
model = keras.models.Sequential()

# layer[0] Flatten layer has role to convert each input
# data to a 1D array.
model.add(keras.layers.Flatten(input_shape=(8,)))  # Specifica l'input_shape come una tupla (10,)

# layer[1] first dense hidden layer with 300 neurons.
# Activation function: ReLU
model.add(keras.layers.Dense(300, activation="relu"))

# layer[2] second dense hidden layer with 100 neurons
# Activation function: ReLU
model.add(keras.layers.Dense(100, activation="relu"))

# layer[3] last dense hidden layer with 10 neurons (one per class)
# Activation function: softmax
model.add(keras.layers.Dense(10, activation="softmax"))

# compile model
model.compile(loss="sparse_categorical_crossentropy",  # we have sparse labels
              optimizer="sgd",
              metrics=["accuracy"])


In [59]:
# training model
history = model.fit(x_train_normalized, y_train, epochs=30, validation_data=(x_valid_normalized,  y_valid))


# plot training history
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

Epoch 1/30
83/83 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.0350 - val_loss: nan - val_accuracy: 0.0333
Epoch 2/30
83/83 [==============================] - 1s 7ms/step - loss: nan - accuracy: 0.0350 - val_loss: nan - val_accuracy: 0.0333
Epoch 3/30
83/83 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0350 - val_loss: nan - val_accuracy: 0.0333
Epoch 4/30
83/83 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0350 - val_loss: nan - val_accuracy: 0.0333
Epoch 5/30
83/83 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0350 - val_loss: nan - val_accuracy: 0.0333
Epoch 6/30
83/83 [==============================] - 1s 7ms/step - loss: nan - accuracy: 0.0350 - val_loss: nan - val_accuracy: 0.0333
Epoch 7/30
83/83 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.0350 - val_loss: nan - val_accuracy: 0.0333
Epoch 8/30
83/83 [==============================] - 0s 5ms/ste